In [1]:
# ADHD Classification Based on Activity
# Purpose: Determine whether activity can predict diagnosis of ADHD
# Author: Alexander Maksiaev

In [11]:
# Housekeeping

import pandas as pd
import numpy as np
import os

main_dir = "c:/Users/maksi/Documents/Statistics/Projects/Movement_Mental_Health/"

adhd_dir = os.path.join(main_dir, "hyperaktiv/")
activity_dir = os.path.join(adhd_dir, "activity_data/")
hrv_dir = os.path.join(adhd_dir, "hrv_data/")
controls_dir = os.path.join(adhd_dir, "hyperaktiv_with_controls/hyperactive_with_controls/")

os.chdir(adhd_dir)

patient_info = pd.read_csv("patient_info.csv", delimiter=";")
patient_info

os.chdir(activity_dir)
activity_data = pd.read_csv("patient_activity_01.csv", delimiter=";")
activity_data

os.chdir(hrv_dir)
hrv_data = pd.read_csv("patient_hr_1.csv", delimiter=";")
hrv_data

,TIMESTAMP,HRV
0,2009-03-04 11:00:00.000,2294.92
1,2009-03-04 11:00:02.295,631.84
2,2009-03-04 11:00:02.927,624.02
3,2009-03-04 11:00:03.551,636.72
4,2009-03-04 11:00:04.188,625.98
...,...,...
107469,2009-03-05 08:07:30.918,671.88
107470,2009-03-05 08:07:31.590,669.92
107471,2009-03-05 08:07:32.260,670.90
107472,2009-03-05 08:07:32.931,662.11
